The purpose of the second experiment is to show the impact of the scaling factor on the results.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from random import choices
from minmax.models import densenet, resnet, vgg

transform_test = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_reverse = transforms.Compose([
    transforms.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
])

testset = torchvision.datasets.ImageNet(
    root='./files/ImageNet',
    split="val",
    transform=transform_test
)

sample = 13899
data, target = testset[sample]

net, r = vgg.vgg19(pretrained=True), float(0.000001)
net.eval()
net.zero_grad()
net.requires_grad_(False)
print("Loading Finished")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from minmax.mm import mmTensor
from minmax.ImageNetClasses import classes
import scipy.ndimage as ndimage
from pytorch_grad_cam import GradCAM
import os

OUTPUT_FOLDER = "Experiment_2"
try:
    os.mkdir(OUTPUT_FOLDER)
    print("Output Folder Created")
except FileExistsError:
    print("Output Folder Exists")

def remove_flags(m):
    if hasattr(m, 'scale_factor'):
        del m.scale_factor
    if hasattr(m, 'debug_range'):
        del m.debug_range
    if hasattr(m, 'debug_relu'):
        del m.debug_relu
    if hasattr(m, 'autoscale_relu'):
        del m.autoscale_relu

net.apply(remove_flags)

# Output Original
image = transform_reverse(data).movedim(0,2).numpy()
plt.clf()
plt.axis("off")
plt.imshow(image)
plt.savefig("{}/Original.png".format(OUTPUT_FOLDER), bbox_inches='tight',pad_inches=0)
plt.clf()

for s in [0.01, 0.1, 1, 5, 10, 20, 40, 80]:
    # Get Minmax Gradient
    def set_scale(m):
        #m.debug_relu = True
        m.autoscale_relu = s / 100
    net.apply(set_scale)
    inputs = mmTensor(data.unsqueeze(0))
    mask = torch.full(data.shape, float(r), requires_grad=True)
    inputs.add_mask(mask, mask)

    outputs = net(inputs)
    prediction = outputs.center().argmax().item()
    one_hot = torch.zeros((1,1000))
    one_hot[0,prediction] = 1

    l = torch.sum((outputs.upper() - outputs.lower()) * one_hot)
    l.backward()

    print("RangeGrad Range: ", outputs.get_range())

    expl_minmax = mask.grad.data.detach().numpy()
    expl_minmax = np.linalg.norm(expl_minmax, axis=0)
    expl_minmax -= expl_minmax.min()
    expl_minmax /= expl_minmax.max()

    # Output Result
    plt.clf()
    plt.axis("off")
    plt.imshow(expl_minmax, cmap=plt.cm.Reds)
    plt.savefig("{}/{}.png".format(OUTPUT_FOLDER, "s{:04d}p".format(int(s * 100))), bbox_inches='tight',pad_inches=0)

    plt.clf()

    fig, axs = plt.subplots(1,2)
    fig.set_figwidth(12)

    axs[0].set_title("Original")
    axs[0].axis("off")
    axs[0].imshow(image)

    axs[1].set_title("RangeGrad (s = {:04d}%/100)".format(int(s * 100)))
    axs[1].axis("off")
    axs[1].imshow(expl_minmax, cmap=plt.cm.Reds)
    plt.show()